<div>
<img src="https://i.ibb.co/v3CvVz9/udd-short.png" width="150"/>
    <br>
    <strong>Universidad del Desarrollo</strong><br>
    <em>Magíster en Data Science</em><br>
    <em>Profesor: Tomás Fontecilla </em><br>

</div>

## <center> **TAREA: Redes Convolucionales** </center>

*02 de Diciembre de 2024*

**Nombre Estudiante**: Cristian Tobar Morales

### **1. Introducción**

En esta tarea, aplicaremos redes neuronales convolucionales (CNN) utilizando la base de datos "Chihuahuas vs Muffin" de Kaggle. Además, compararemos los resultados obtenidos con un modelo de perceptrón multicapa (MLP) para determinar cuál de los dos modelos ofrece una mejor precisión en la clasificación de imágenes.

### **2. Objetivo**

El objetivo es analizar y evaluar el rendimiento de ambos enfoques en términos de precisión y eficacia, proporcionando una visión clara sobre sus fortalezas y debilidades en la tarea de clasificación.

### **3. Metodología**

- Lectura y pre-procesamiento del Conjunto de Datos.
- Visualizando Imágenes
-	Perceptrón Multicapa para Clasificación de Imágenes
-	Redes Neuronales Convolucionales
-	Conclusión 


#### **Librerías requeridas**

In [1]:
import keras
import zipfile
import io
import tensorflow as tf
import pandas as pd
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image, UnidentifiedImageError
from keras.datasets import mnist # type: ignore
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical # type: ignore
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense 
from tensorflow.keras.optimizers import SGD

#### **Lectura y pre-procesamiento del Conjunto de Datos**

La lectura de los sets de entrenamiento y validacion.

Antes de entrenar el modelo debemos:
- Normalizar los valores de los píxiles de los datos en el mismo rango (0,1).
- División del conjunto de datos (Prueba y entrenamiento). 

In [2]:
# Ruta del archivo ZIP 
zip_path = '../Redes-Convolucionales/data/datos.zip' 
extract_path = '../Redes-Convolucionales/data'
# Abre el archivo Zip
with zipfile.ZipFile(zip_path, 'r') as zip_ref: 
    zip_ref.extractall(extract_path)    
    
# Rutas de los datos
train_dir = '../Redes-Convolucionales/data/train'
test_dir = '../Redes-Convolucionales/data/test'

Cargar el conjunto de datos de entrenamiento y prueba. 

- **IMAGEN_SIZE**: Se implementa un tamaño de 128x128 píxeles, que funciona bien con muchas arquitecturas de modelos.
- **BATCH_SIZE**: Define el número de muestras que se procesan antes de que el modelo actualice sus pesos, se utilizará un número de 64.

- **ImageDataGenerator**: Para escalar los píxeles de las imágenes y aplicar aumentos como volteos verticales y horizontales. Esto ayudará a mejorar la generalización de tu modelo al introducir variaciones en las imágenes de entrenamiento.

In [ ]:
import os
BATCH_SIZE = 64 
IMAGEN_SIZE = (128, 128) 


def verify_images(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                with Image.open(file_path) as img:
                    img.verify()  # Verificar que el archivo es una imagen válida
            except (UnidentifiedImageError, IOError):
                os.remove(file_path)  # Eliminar el archivo no válido

# Verificar imágenes en los directorios de entrenamiento y prueba
verify_images(train_dir)
verify_images(test_dir)


# Crear un generador de datos
data_generator = ImageDataGenerator(rescale=1./255, # Normalizar las imágenes
                                    rotation_range=20,  # Rotar las imágenes aleatoriamente dentro de un rango de 20 grados
                                    zoom_range=0.2,  # Aplicar zoom aleatorio a las imágenes
                                    fill_mode='nearest',  # Utilizar 'nearest' para completar píxeles faltantes tras transformaciones
                                    channel_shift_range=0.1,  # Desplazamiento aleatorio de los canales de color
                                    vertical_flip=True, 
                                    horizontal_flip=True)

# Cargar el conjunto de datos de entrenamiento
train_dataset = data_generator.flow_from_directory(
    train_dir,
    target_size=IMAGEN_SIZE,  
    batch_size=BATCH_SIZE,
    class_mode='binary' #Clasificación binaria
)

# Cargar el conjunto de datos de prueba
test_dataset = data_generator.flow_from_directory(
    test_dir,
    target_size=IMAGEN_SIZE,  
    batch_size=BATCH_SIZE,
    class_mode='binary' 
)